In [38]:
import csv
from pydriller import Repository

def analyze_repo(repo_url, output_csv):
    with open(output_csv, mode='w', newline='') as csv_file:
        fieldnames = ['Commit SHA', 'Java File', 'Old function signature', 'New function signature']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        for commit in Repository(repo_url).traverse_commits():
            for modification in commit.modified_files:
                if modification.filename.endswith('.java'):
                    old_content = modification.source_code_before
                    new_content = modification.source_code

                    if old_content and new_content:
                        old_lines = old_content.split('\n')
                        new_lines = new_content.split('\n')

                        for old_line, new_line in zip(old_lines, new_lines):
                            old_signature = old_line.strip()
                            new_signature = new_line.strip()

                            if '(' in old_signature and '(' in new_signature:
                                old_signature = old_signature.split('(', 1)[1].split(')', 1)[0].strip()
                                new_signature = new_signature.split('(', 1)[1].split(')', 1)[0].strip()

                                if len(old_signature.split(',')) < len(new_signature.split(',')):
                                    writer.writerow({'Commit SHA': commit.hash,
                                                     'Java File': modification.filename,
                                                     'Old function signature': old_signature,
                                                     'New function signature': new_signature})

if __name__ == "__main__":
    analyze_repo('https://github.com/google/guava', 'output.csv')
